In [1]:
from PYME.LMVis import VisGUI

%gui wx

ERROR:PYME.recipes.modules:Error loading plugin: cc_drift_cor.plugins.recipes.chung_cc
Traceback (most recent call last):
  File "c:\users\zrc4\code\python-microscopy\PYME\recipes\modules.py", line 42, in <module>
    m = __import__(mn, fromlist=mn.split('.')[:-1])
ModuleNotFoundError: No module named 'cc_drift_cor'
ERROR:PYME.recipes.modules:Error loading plugin: cc_drift_cor.plugins.recipes.processing
Traceback (most recent call last):
  File "c:\users\zrc4\code\python-microscopy\PYME\recipes\modules.py", line 42, in <module>
    m = __import__(mn, fromlist=mn.split('.')[:-1])
ModuleNotFoundError: No module named 'cc_drift_cor'
ERROR:PYME.recipes.modules:Error loading plugin: pyme_4pi.recipe_modules.conversion
Traceback (most recent call last):
  File "c:\users\zrc4\code\python-microscopy\PYME\recipes\modules.py", line 42, in <module>
    m = __import__(mn, fromlist=mn.split('.')[:-1])
ModuleNotFoundError: No module named 'pyme_4pi'
ERROR:PYME.recipes.modules:Error loading plugin: cc

Trying to load 3rd party recipe module cc_drift_cor.plugins.recipes.chung_cc
Trying to load 3rd party recipe module cc_drift_cor.plugins.recipes.processing
Trying to load 3rd party recipe module ch_shrinkwrap.recipe_modules.surface_fitting
Loaded 3rd party recipe module ch_shrinkwrap.recipe_modules.surface_fitting
Trying to load 3rd party recipe module ch_shrinkwrap.recipe_modules.surface_feature_extraction
Loaded 3rd party recipe module ch_shrinkwrap.recipe_modules.surface_feature_extraction
Trying to load 3rd party recipe module pyme_4pi.recipe_modules.conversion
Trying to load 3rd party recipe module cc_drift_cor.plugins.recipes.localisations
Trying to load 3rd party recipe module cc_drift_cor.plugins.recipes.io
Trying to load 3rd party recipe module oidic.recipe_modules.reconstruct
Loaded 3rd party recipe module oidic.recipe_modules.reconstruct


INFO:OpenGL.GLUT.special:Using NT-specific GLUT calls with exit callbacks


Detected a .git folder, assuming a development install
quaternion module not found, disabling custom clip plane orientations
quaternion module not found, disabling custom clip plane orientations


In [2]:
pymevis = VisGUI.ipython_pymevisualize()
pipeline = pymevis.pipeline

import numpy as np
import os

DEBUG:PYME.LMVis.visCore:Creating VisGUI menu bar
DEBUG:PYME.LMVis.Extras:Initializing QPobjectSegment plugin
DEBUG:PYME.LMVis.Extras:Initializing animation plugin
DEBUG:PYME.LMVis.Extras:Initializing clusterAnalysis plugin
DEBUG:PYME.LMVis.Extras:Initializing dockedPanel plugin
DEBUG:PYME.LMVis.Extras:Initializing extra_layers plugin
DEBUG:PYME.LMVis.Extras:Initializing fiducial plugin
DEBUG:PYME.LMVis.Extras:Initializing flagZMotionArtifacts plugin
DEBUG:root:Adding menu items for event filters
DEBUG:PYME.LMVis.Extras:Initializing fusionUtils plugin
DEBUG:PYME.LMVis.Extras:Initializing multiColourAnalysis plugin
DEBUG:PYME.LMVis.Extras:Initializing multiviewMapping plugin
DEBUG:root:Adding menu items for multi-view manipulation
DEBUG:PYME.LMVis.Extras:Initializing objectMeasurements plugin
DEBUG:PYME.LMVis.Extras:Initializing particleTracking plugin
DEBUG:PYME.LMVis.Extras:Initializing perFrameVariable plugin
DEBUG:PYME.LMVis.Extras:Initializing photophysics plugin
DEBUG:PYME.LMVis.E

New Canvas
Saved Views are deprecated -  use the ones with animation instead and remove me!


ERROR:PYME.LMVis.Extras:Error loading plugin: pyme_4pi.visgui_modules.precision
Traceback (most recent call last):
  File "c:\users\zrc4\code\python-microscopy\PYME\LMVis\Extras\__init__.py", line 46, in InitPlugins
    m = __import__(mn, fromlist=mn.split('.')[:-1])
ModuleNotFoundError: No module named 'pyme_4pi'
ERROR:PYME.LMVis.Extras:Error loading plugin: pyme_4pi.visgui_modules.conversion
Traceback (most recent call last):
  File "c:\users\zrc4\code\python-microscopy\PYME\LMVis\Extras\__init__.py", line 46, in InitPlugins
    m = __import__(mn, fromlist=mn.split('.')[:-1])
ModuleNotFoundError: No module named 'pyme_4pi'


Creating fold panel
GenPanels
Creating datasource panel
RecipeView._layout
Input datasources:  []
Image list size: 1
Creating layers GUI
height:  0
NB best size: wx.Size(200, 0)
RecipeView._layout


RuntimeError: wrapped C/C++ object of type foldPanel has been deleted

Input datasources:  []
recipe.execute()
RecipeView._layout


RuntimeError: wrapped C/C++ object of type foldPanel has been deleted

Input datasources:  []
0
OpenGL - Version: b'4.6.0 NVIDIA 442.50'
Shader - Version: b'4.60 NVIDIA'
GL_MAX_SAMPLES: 32, GL_SAMPLES: 4


DEBUG:PYME.LMVis.shader_programs.ShaderProgramFactory:New shader program created: <class 'PYME.LMVis.shader_programs.DefaultShaderProgram.DefaultShaderProgram'>


Ev Idle
('points', 't')
refreshed


In [3]:
"""
This compares ICTM mesh quality as a function of parameters.
"""

# Where to save the intermediate files generated 
save_fp = 'C:\\Users\\zrc4\\Downloads\\test_ictm_vs_psr'
#save_fp = '/Users/zrc4/Downloads/test_ictm_vs_psr'

if not os.path.exists(save_fp):
    os.mkdir(save_fp)

# three way junction generation parameters
centroid = np.array([0,0,0])
h, r = 500, 50       # capsule length, capsule radius
smoothing = r      # ~h/2 creates a three-way junction with a "sheet" (but it's puffy like seaweed)
# loc precision
psf_width = 250.0
mean_photon_count = 300.0

# octree-based reconstruction parameters
cull_inner_surfaces = True
n_points_min = 10
remesh = True
repair = False
smooth_curvature = True
density = 9e-7

# ictm reconstruction parameters
ictm_max_iters = np.hstack([5, np.logspace(1,2,10).astype(int)])  # integer
ictm_step_size = np.logspace(0,2,10)                              # float
ictm_search_rad = np.logspace(0,2,10)                             # float
ictm_remesh_every = np.array([1,5,10,20,50,100])                  # integer
ictm_search_k = 20                                                # integer

n_iters = len(ictm_max_iters)*len(ictm_step_size)*len(ictm_search_rad)*len(ictm_remesh_every)
print(f"Pre-stride ictm iterations: {n_iters}")

stride = 8

ictm_max_iters = ictm_max_iters[::stride]
ictm_step_size = ictm_step_size[::stride]
ictm_search_rad = ictm_search_rad[::stride]
ictm_remesh_every = ictm_remesh_every[::stride]

n_iters = len(ictm_max_iters)*len(ictm_step_size)*len(ictm_search_rad)*len(ictm_remesh_every)
print(f"Post-stride ictm iterations: {n_iters}")

# screened poisson reconstruction parameters
spr_fulldepth=5
spr_max_iters = np.arange(7,10)            # integer
spr_depth = 2**np.arange(1,4)              # integer
spr_search_k = np.arange(5,30,5)           # integer
spr_scale = [1.1] #np.linspace(0,1.2,5)    # float
spr_samplespernode=np.linspace(13,19,5)    # float
spr_pointweight=np.linspace(0,4,5)         # float

n_iters = len(spr_max_iters)*len(spr_depth)*len(spr_search_k)*len(spr_scale)*len(spr_samplespernode)*len(spr_pointweight)
print(f"Pre-stride spr iterations: {n_iters}")

stride = 3

spr_max_iters = spr_max_iters[::stride]
spr_depth = spr_depth[::stride]
spr_search_k = spr_search_k[::stride]
spr_samplespernode = spr_samplespernode[::stride]
spr_pointweight = spr_pointweight[::stride]

n_iters = len(spr_max_iters)*len(spr_depth)*len(spr_search_k)*len(spr_scale)*len(spr_samplespernode)*len(spr_pointweight)
print(f"Post-stride spr iterations: {n_iters}")

Pre-stride ictm iterations: 6600
Post-stride ictm iterations: 8
Pre-stride spr iterations: 1125
Post-stride spr iterations: 8


In [4]:
"""
Generate three-way junction
"""

from ch_shrinkwrap.shape import ThreeWayJunction

twj = ThreeWayJunction(h, r, centroid, smoothing)

In [5]:
"""
Generate and save three-way junction ponts
"""
import time

from ch_shrinkwrap import util

cap_points = twj.points(p=0.0001, psf_width=psf_width, 
                        mean_photon_count=mean_photon_count, 
                        resample=True)
cap_sigma = twj._sigma

no, scale = 0.1, 1.2
bbox = [np.min(cap_points[:,0]), np.min(cap_points[:,1]), 
        np.min(cap_points[:,2]), np.max(cap_points[:,0]),
        np.max(cap_points[:,1]), np.max(cap_points[:,2])]
bbox = [scale*x for x in bbox]
xl, yl, zl, xu, yu, zu = bbox
xn, yn, zn = xu-xl, yu-yl, zu-zl
ln = int(no*len(cap_points)/(1.0-no))
noise_points = np.random.rand(ln,3)*(np.array([xn,yn,zn])[None,:]) \
               + (np.array([xl,yl,zl])[None,:])
noise_sigma = util.loc_error(noise_points.shape, model='poisson', 
                         psf_width=psf_width, 
                         mean_photon_count=mean_photon_count)

points = np.vstack([cap_points,noise_points])
sigma = np.vstack([cap_sigma,noise_sigma])
s = np.sqrt((sigma*sigma).sum(1))

points_time = time.strftime('%Y%d%m_%HH%M')
points_fn = f"twj_h{h}_r{r}_smoothing{smoothing}_{points_time}".replace('.','_')+".txt"
# points_fn = "twj_h500_r50_smoothing50_20210911_22H12.txt"
points_fp = os.path.join(save_fp, points_fn)
np.savetxt(points_fp, np.vstack([points.T,s]).T, header="x y z sigma")

In [6]:
# Now we need PYMEVis
pymevis.OpenFile(points_fp)


Creating Pipeline
invalid_raise: True
RecipeView._layout


RuntimeError: wrapped C/C++ object of type foldPanel has been deleted

Input datasources:  ['FitResults']
RecipeView._layout


SystemError: <class 'wx._core.WindowDestroyEvent'> returned a result with an error set

RuntimeError: wrapped C/C++ object of type foldPanel has been deleted

...

Input datasources:  ['FitResults']
0


DEBUG:PYME.LMVis.VisGUI:Finished creating tabs


Gui stuff done
lw update
Creating layers GUI
hl update
height:  320
NB best size: wx.Size(222, 320)


DEBUG:PYME.LMVis.visCore:Added layer, datasouce=output
DEBUG:PYME.LMVis.shader_programs.ShaderProgramFactory:New shader program created: <class 'PYME.LMVis.shader_programs.DefaultShaderProgram.OpaquePointShaderProgram'>


lw update
hl update
hl update


In [8]:
# Octree renderer (original mesh based on parameters above)
from PYME.LMVis.Extras.extra_layers import gen_octree_from_points
from PYME.recipes.surface_fitting import DualMarchingCubes

oc_name = gen_octree_from_points(pymevis)
surf_name, surf_count = pipeline.new_ds_name('surf', return_count=True)

recipe = pipeline.recipe
dmc = DualMarchingCubes(recipe, invalidate_parent=False, input=oc_name, 
                        output=surf_name, threshold_density=density, 
                        n_points_min=n_points_min, remesh=remesh,
                        repair=repair, 
                        cull_inner_surfaces=cull_inner_surfaces, 
                        smooth_curvature=smooth_curvature)
recipe.add_modules_and_execute([dmc,])

octree_fp = points_fp.split('.txt')[0] + '_octree.stl'
pipeline.dataSources[dmc.output].to_stl(octree_fp)

rmin:[-705.4178  -903.26794 -346.81973], r_max:[710.7576  747.06116 356.93292]
Octree  (octree0) created
recipe.execute()
Executing <PYME.recipes.surface_fitting.DualMarchingCubes object at 0x00000223027A6CA8>
Dual grid created in 0.131 s
March in 0.001 s
Generating TriangularMesh object
Data munged to vertices, faces
initializing halfedges ...
vertices.shape = (277, 3), faces.shape = (550, 3)
iterating edges
done initializing halfedges
Generated TriangularMesh object
Split count: 216
Flip count: 173 [0 failed]
Flip count: 9 [0 failed]
Flip count: 1 [0 failed]
Flip count: 0 [0 failed]
Collapse count: 266[2 failed]
Flip count: 68 [0 failed]
Flip count: 3 [0 failed]
Flip count: 0 [0 failed]
Split count: 152
Flip count: 110 [0 failed]
Flip count: 7 [0 failed]
Flip count: 1 [0 failed]
Flip count: 0 [0 failed]
Collapse count: 173[1 failed]
Flip count: 48 [0 failed]
Flip count: 4 [0 failed]
Flip count: 0 [0 failed]
Split count: 113
Flip count: 91 [0 failed]
Flip count: 5 [0 failed]
Flip coun

SystemError: <class 'wx._core.WindowDestroyEvent'> returned a result with an error set

SystemError: <class 'wx._core.WindowDestroyEvent'> returned a result with an error set

SystemError: <class 'wx._core.WindowDestroyEvent'> returned a result with an error set

RuntimeError: wrapped C/C++ object of type foldPanel has been deleted

...

SystemError: <class 'wx._core.WindowDestroyEvent'> returned a result with an error set

SystemError: <class 'wx._core.WindowDestroyEvent'> returned a result with an error set

SystemError: <class 'wx._core.WindowDestroyEvent'> returned a result with an error set

RuntimeError: wrapped C/C++ object of type foldPanel has been deleted

Input datasources:  ['FitResults', 'octree0']


In [9]:
"""
At this point, we will start from the original surface and iterate using
different parameters.
"""
# Shrinkwrap renderer
from ch_shrinkwrap import _membrane_mesh as membrane_mesh

# max_iters = np.array([10])
# remesh_every = np.array([5])
# step_size = np.array([12.9])
# search_k = np.array([20])

failed_count = 0
for it in ictm_max_iters:
    for lam in ictm_step_size:
        for sr in ictm_search_rad:
            for re in ictm_remesh_every:
                # Copy the mesh over
                mesh = membrane_mesh.MembraneMesh(mesh=pipeline.dataSources[dmc.output])

                # set params
                mesh.max_iter = it
                mesh.step_size = lam
                mesh.search_k = ictm_search_k
                mesh.search_rad = sr
                mesh.remesh_frequency = re

                start = time.time()
                failed = False
                try:
                    mesh.shrink_wrap(points, s, method='conjugate_gradient')
                except:
                    failed_count += 1
                    failed=True
                stop = time.time()
                duration = stop-start
                if not failed:
                    wrap_fp = points_fp.split('.txt')[0] + "_".join(f"_iters{it}_remesh{re}_lambda{lam:.1f}_searchk{ictm_search_k}_searchrad{sr:.1f}_noise{no:.1f}_ntriangles{mesh.faces.shape[0]}_duration{duration:.1f}_ictm".split('.')) + ".stl"
                    mesh.to_stl(wrap_fp)
print(f'# failed: {failed_count}')

Test Statistic 0.973610
Area ratio is 0.0462
Split count: 193
Flip count: 129 [0 failed]
Flip count: 11 [0 failed]
Flip count: 1 [0 failed]
Flip count: 0 [0 failed]
Collapse count: 142[0 failed]
Flip count: 54 [0 failed]
Flip count: 5 [0 failed]
Flip count: 0 [0 failed]
Split count: 76
Flip count: 62 [0 failed]
Flip count: 3 [0 failed]
Flip count: 0 [0 failed]
Collapse count: 90[0 failed]
Flip count: 34 [0 failed]
Flip count: 3 [0 failed]
Flip count: 0 [0 failed]
Split count: 74
Flip count: 65 [0 failed]
Flip count: 5 [0 failed]
Flip count: 0 [0 failed]
Collapse count: 79[0 failed]
Flip count: 38 [0 failed]
Flip count: 4 [0 failed]
Flip count: 0 [0 failed]
Split count: 94
Flip count: 80 [0 failed]
Flip count: 2 [0 failed]
Flip count: 1 [0 failed]
Flip count: 0 [0 failed]
Collapse count: 112[0 failed]
Flip count: 37 [0 failed]
Flip count: 0 [0 failed]
Split count: 82
Flip count: 72 [0 failed]
Flip count: 2 [0 failed]
Flip count: 0 [0 failed]
Collapse count: 79[1 failed]
Flip count: 31 [

c:\users\zrc4\code\ch-shrinkwrap\ch_shrinkwrap\conj_grad.py:148: RuntimeWarning: invalid value encountered in float_scalars
  / (np.linalg.norm(S[:,pair[0]])*np.linalg.norm(S[:,pair[1]]))))


Test Statistic 0.910983
Area ratio is 0.0530
Split count: 13
Flip count: 14 [0 failed]
Flip count: 1 [0 failed]
Flip count: 0 [0 failed]
Collapse count: 31[0 failed]
Flip count: 15 [0 failed]
Flip count: 0 [0 failed]
Split count: 28
Flip count: 28 [0 failed]
Flip count: 0 [0 failed]
Collapse count: 33[0 failed]
Flip count: 10 [0 failed]
Flip count: 0 [0 failed]
Split count: 40
Flip count: 35 [0 failed]
Flip count: 0 [0 failed]
Collapse count: 51[0 failed]
Flip count: 12 [0 failed]
Flip count: 0 [0 failed]
Split count: 79
Flip count: 59 [0 failed]
Flip count: 3 [0 failed]
Flip count: 0 [0 failed]
Collapse count: 81[0 failed]
Flip count: 22 [0 failed]
Flip count: 2 [0 failed]
Flip count: 0 [0 failed]
Split count: 75
Flip count: 52 [0 failed]
Flip count: 2 [0 failed]
Flip count: 1 [0 failed]
Flip count: 0 [0 failed]
Collapse count: 74[0 failed]
Flip count: 20 [0 failed]
Flip count: 0 [0 failed]
Target mean length: 81.52721543224425   Resulting mean length: 86.9556
Test Statistic 0.811531


In [10]:
"""
Now calculate SPR.
"""

def screened_poisson(points_fp, rowstoskip=1, strformat='X Y Z Reflectance', separator='SPACE',
                     colorformat='[0-255]', onerror='skip', k=10, smoothiter=0,
                     flipflag=False, viewpos=[0,0,0], depth=8, fulldepth=5,
                     cgdepth=0, scale=1.1, samplespernode=1.5, pointweight=4, 
                     iters=8, confidence=False, preclean=False):
    """
    Run screened poisson reconstruction on a set of points, using meshlab.
    
    For more information on these parameters, see meshlab.
    
    Parameters
    ----------
    points_fp : str
        Path to text file containing a point cloud represented as a set of XYZ coordinates
    see meshlab
    
    Returns
    -------
    str
        File path to STL of reconstruction
        
    """
    import pymeshlab as ml

    ms = ml.MeshSet()  # create a mesh
    ms.load_new_mesh(file_name=points_fp,
                     rowtoskip=rowstoskip,
                     strformat=strformat,
                     separator=separator,
                     rgbmode=colorformat,
                     onerror=onerror)  # load points
    start = time.time()
    # compute normals
    ms.compute_normals_for_point_sets(k=k,  # number of neighbors
                                      smoothiter=smoothiter,
                                      flipflag=flipflag,
                                      viewpos=viewpos)
    # run SPR
    ms.surface_reconstruction_screened_poisson(visiblelayer=False,
                                               depth=depth,
                                               fulldepth=spr_fulldepth,
                                               cgdepth=cgdepth,
                                               scale=scale,
                                               samplespernode=samplespernode,
                                               pointweight=pointweight,
                                               iters=iters,
                                               confidence=confidence,
                                               preclean=preclean)
    stop = time.time()
    duration = stop-start
    # save surface
    surface_fp = points_fp.split('.txt')[0] + "_".join(f"_searchk{k}_depth{depth}_scale{scale:.1f}_samplespernode{samplespernode:.1f}_pointweight{pointweight:.1f}_iters{iters}_noise{no:.1f}_duration{duration:.1f}_spr".split('.')) + ".stl"
    ms.save_current_mesh(file_name=surface_fp, unify_vertices=True)
    
    return surface_fp

# max_iters = np.array([8],dtype=int)
# depth = np.array([8],dtype=int)
# search_k = np.array([10])
# scale=np.array([1.1])
# samplespernode=np.array([1.5])
# pointweight=np.array([4])

for it in spr_max_iters:
    for k in spr_search_k:
        for d in spr_depth:
            for sc in spr_scale:
                for spn in spr_samplespernode:
                    for wt in spr_pointweight:
                        spr_fp = screened_poisson(points_fp, k=k, depth=d, fulldepth=spr_fulldepth, 
                                                  scale=sc, samplespernode=spn, pointweight=wt, iters=it)

In [11]:
"""
Now load and analyze the meshes.
"""

import re
import glob
#from ch_shrinkwrap import _membrane_mesh as membrane_mesh
from ch_shrinkwrap import evaluation_utils

In [12]:
# Generate a set of test points from the SDF for comparison
dx_min, p = 5, 1.0
test_points, test_normals = twj.points(density=1.0/(dx_min**3), p=1.0, 
                                       psf_width=psf_width, 
                                       mean_photon_count=mean_photon_count, 
                                       resample=True, noise=None, 
                                       return_normals=True)
print(test_points.shape, test_normals.shape)

(248280, 3) (248280, 3)


In [15]:
def test_points_mesh_stats(points, normals, mesh):
    # Generate a set of test points from this mesh
    mesh_points, mesh_normals = evaluation_utils.points_from_mesh(mesh, 
                                                                  dx_min=dx_min, 
                                                                  p=p, return_normals=True)
    
    # Compute ordered points between this mesh and test_points
    ox, oa, mx, ma = evaluation_utils.construct_ordered_pairs(points, mesh_points, 
                                             normals, mesh_normals, 
                                             dx_max=dx_min)
    
    print(test_points.shape[0], mesh_points.shape[0], ox.shape[0], mx.shape[0])

    # Compute hausdorff and mean distance (nm) and smoothness (rad)
    hd, md = evaluation_utils.mean_and_hausdorff_distance_from_ordered_pairs(points, mesh_points, ox, oa, mx, ma)
    ha, aa = evaluation_utils.mean_and_hausdorff_smoothness_from_ordered_pairs(normals, mesh_normals, ox, oa, mx, ma)
    
    return hd, md, ha, aa

In [16]:
ictm_its, ictm_remesh, ictm_lamb, ictm_pd = [], [], [], []
ictm_searchk, ictm_noise, ictm_ntris, ictm_runtime = [], [], [], []
ictm_searchrad = []

ictm_error, ictm_hausdorff_dist, ictm_hausdorff_ang = [], [], []
ictm_mean_dist, ictm_mean_ang = [], []

failed = 0
for fn in glob.glob(save_fp+"\\*ictm.stl"):
    try:
        print(f"Loading {fn}")
        mesh = membrane_mesh.MembraneMesh.from_stl(fn)
        ictm_its.append(int(re.search("(?<=iters)\d+",fn).group(0)))
        ictm_remesh.append(int(re.search("(?<=remesh)\d+",fn).group(0)))
        ictm_lamb.append(float(re.search("(?<=lambda)\d+\_\d+",fn).group(0).replace('_','.')))
        ictm_searchk.append(int(re.search("(?<=searchk)\d+",fn).group(0)))
        ictm_searchrad.append(float(re.search("(?<=searchrad)\d+\_\d+",fn).group(0).replace('_','.')))
        ictm_noise.append(float(re.search("(?<=noise)\d+\_\d+",fn).group(0).replace('_','.')))
        ictm_ntris.append(int(re.search("(?<=ntriangles)\d+",fn).group(0)))
        ictm_runtime.append(float(re.search("(?<=duration)\d+\_\d+",fn).group(0).replace('_','.')))

        # Calculate error per face
        vecs = mesh._vertices[mesh.faces]['position']
        ictm_error.append(twj.sdf(vecs.mean(1).T))
        
        # Calculate distance and angle stats
        hd, md, ha, ma = test_points_mesh_stats(test_points, test_normals, mesh)
        ictm_hausdorff_dist.append(hd)
        ictm_hausdorff_ang.append(ha)
        ictm_mean_dist.append(md)
        ictm_mean_ang.append(ma)
        
    except:
        failed += 1
print(f"# failed: {failed}")
    
ictm_its, ictm_remesh = np.array(ictm_its), np.array(ictm_remesh)
ictm_lamb, ictm_searchk = np.array(ictm_lamb), np.array(ictm_searchk)
ictm_noise, ictm_ntris = np.array(ictm_noise), np.array(ictm_ntris)
ictm_runtime, ictm_hausdorff_dist = np.array(ictm_runtime), np.array(ictm_hausdorff_dist)
ictm_hausdorff_ang, ictm_mean_dist = np.array(ictm_hausdorff_ang), np.array(ictm_mean_dist)
ictm_mean_ang, ictm_searchrad = np.array(ictm_mean_ang), np.array(ictm_searchrad)

Loading C:\Users\zrc4\Downloads\test_ictm_vs_psr\twj_h500_r50_smoothing50_20220601_17H21_iters59_remesh1_lambda1_0_searchk20_searchrad1_0_noise0_1_ntriangles264_duration4_5_ictm.stl
Data munged to vertices, faces
initializing halfedges ...
vertices.shape = (134, 3), faces.shape = (264, 3)
iterating edges
done initializing halfedges
248280 231376 7955 6584
Loading C:\Users\zrc4\Downloads\test_ictm_vs_psr\twj_h500_r50_smoothing50_20220601_17H21_iters59_remesh1_lambda1_0_searchk20_searchrad59_9_noise0_1_ntriangles218_duration2_3_ictm.stl
Data munged to vertices, faces
initializing halfedges ...
vertices.shape = (111, 3), faces.shape = (218, 3)
iterating edges
done initializing halfedges
248280 210080 5494 4004
Loading C:\Users\zrc4\Downloads\test_ictm_vs_psr\twj_h500_r50_smoothing50_20220601_17H21_iters5_remesh1_lambda1_0_searchk20_searchrad1_0_noise0_1_ntriangles18338_duration7_7_ictm.stl
Data munged to vertices, faces
initializing halfedges ...
vertices.shape = (9171, 3), faces.shape = 

In [18]:
spr_searchk, spr_depth, spr_scale, spr_pd = [], [], [], []
spr_spn, spr_pointweight, spr_its, spr_noise = [], [], [], []
spr_runtime = []

spr_error, spr_hausdorff_dist, spr_hausdorff_ang = [], [], []
spr_mean_dist, spr_mean_ang = [], []

fail_count = 0
for fn in glob.glob(save_fp+"\\*spr.stl"):
    try:
        print(fn)
        mesh = membrane_mesh.MembraneMesh.from_stl(fn)
        spr_searchk.append(int(re.search("(?<=searchk)\d+",fn).group(0)))
        spr_depth.append(int(re.search("(?<=depth)\d+",fn).group(0)))
        spr_scale.append(float(re.search("(?<=scale)\d+\_\d+",fn).group(0).replace('_','.')))
        spr_spn.append(float(re.search("(?<=samplespernode)\d+\_\d+",fn).group(0).replace('_','.')))
        spr_pointweight.append(float(re.search("(?<=pointweight)\d+_\d+",fn).group(0).replace('_','.')))
        spr_its.append(int(re.search("(?<=iters)\d+",fn).group(0)))
        spr_noise.append(float(re.search("(?<=noise)\d+\_\d+",fn).group(0).replace('_','.')))
        spr_runtime.append(float(re.search("(?<=duration)\d+\_\d+",fn).group(0).replace('_','.')))

         # Calculate error per face
        vecs = mesh._vertices[mesh.faces]['position']
        spr_error.append(twj.sdf(vecs.mean(1).T))
        
        # Calculate distance and angle stats
        hd, md, ha, ma = test_points_mesh_stats(test_points, test_normals, mesh)
        spr_hausdorff_dist.append(hd)
        spr_hausdorff_ang.append(ha)
        spr_mean_dist.append(md)
        spr_mean_ang.append(ma)
        
    except(IndexError):
        print("index error")
        fail_count +=1
        
print(f'# failed: {fail_count}')
    
spr_searchk = np.array(spr_searchk)
spr_depth, spr_scale = np.array(spr_depth), np.array(spr_scale)
spr_spn, spr_pointweight = np.array(spr_spn), np.array(spr_pointweight)
spr_its, spr_noise = np.array(spr_its), np.array(spr_noise)
spr_runtime, spr_hausdorff_dist = np.array(spr_runtime), np.array(spr_hausdorff_dist)
spr_hausdorff_ang, spr_mean_dist = np.array(spr_hausdorff_ang), np.array(spr_mean_dist)
spr_mean_ang = np.array(spr_mean_ang)

C:\Users\zrc4\Downloads\test_ictm_vs_psr\twj_h500_r50_smoothing50_20220601_17H21_searchk20_depth2_scale1_1_samplespernode13_0_pointweight0_0_iters7_noise0_1_duration0_1_spr.stl
Data munged to vertices, faces
initializing halfedges ...
vertices.shape = (42, 3), faces.shape = (62, 3)
iterating edges
done initializing halfedges
248280 65984 480 373
C:\Users\zrc4\Downloads\test_ictm_vs_psr\twj_h500_r50_smoothing50_20220601_17H21_searchk20_depth2_scale1_1_samplespernode13_0_pointweight3_0_iters7_noise0_1_duration0_1_spr.stl
Data munged to vertices, faces
initializing halfedges ...
vertices.shape = (43, 3), faces.shape = (61, 3)
iterating edges
done initializing halfedges
248280 113840 721 549
C:\Users\zrc4\Downloads\test_ictm_vs_psr\twj_h500_r50_smoothing50_20220601_17H21_searchk20_depth2_scale1_1_samplespernode17_5_pointweight0_0_iters7_noise0_1_duration0_1_spr.stl
Data munged to vertices, faces
initializing halfedges ...
vertices.shape = (43, 3), faces.shape = (62, 3)
iterating edges
done

In [22]:
ictm_mses = []
spr_mses = []
for i in range(len(ictm_error)):
    ictm_mses.append(np.nansum(ictm_error[i]**2)/len(ictm_error[i]))
for i in range(len(spr_error)):
    spr_mses.append(np.nansum(spr_error[i]**2)/len(spr_error[i]))
    
ictm_mses = np.array(ictm_mses)
spr_mses = np.array(spr_mses)

In [25]:
idx = np.argmin(ictm_mean_dist + ictm_hausdorff_dist + ictm_mean_ang + ictm_hausdorff_ang)
print("Best ICTM mesh...")
ictm_fn = points_fp.split('.txt')[0] + "_".join(f"_iters{ictm_its[idx]}_remesh{ictm_remesh[idx]}_lambda{ictm_lamb[idx]:.1f}_searchk{ictm_searchk[idx]}_searchrad{ictm_searchrad[idx]}_noise{ictm_noise[idx]:.1f}_ntriangles{ictm_ntris[idx]}_duration{ictm_runtime[idx]:.1f}_ictm".split('.')) + ".stl"
print(ictm_fn)
print(f"MSE : {ictm_mses[idx]} ",
      f"iters: {ictm_its[idx]}  ", 
      f"remesh frequency: {ictm_remesh[idx]} ",
      f"lambda: {ictm_lamb[idx]}  ", 
      f"search k: {ictm_searchk[idx]}  ", 
      f"noise : {ictm_noise[idx]} ",
      f"# triangles: {ictm_ntris[idx]}", 
      f"duration: {ictm_runtime[idx]}")

idx = np.argmin(spr_mean_dist + spr_hausdorff_dist + spr_mean_ang + spr_hausdorff_ang)
print("Best SPR mesh...")
spr_fn = points_fp.split('.txt')[0] + "_".join(f"_searchk{spr_searchk[idx]}_depth{spr_depth[idx]}_scale{spr_scale[idx]:.1f}_samplespernode{spr_spn[idx]:.1f}_pointweight{spr_pointweight[idx]:.1f}_iters{spr_its[idx]}_noise{spr_noise[idx]:.1f}_duration{spr_runtime[idx]:.1f}_spr".split('.')) + ".stl"
print(spr_fn)
print(f"MSE : {spr_mses[idx]} ",
      f"search k: {spr_searchk[idx]}  ", 
      f"depth: {spr_depth[idx]}  ", 
      f"scale: {spr_scale[idx]}  ",
      f"samples per node: {spr_spn[idx]}  ",
      f"pointweight: {spr_pointweight[idx]}  ",      
      f"iters: {spr_its[idx]}  ",  
      f"noise : {spr_noise[idx]} ",
      f"duration: {spr_runtime[idx]}")

Best ICTM mesh...
C:\Users\zrc4\Downloads\test_ictm_vs_psr\twj_h500_r50_smoothing50_20220601_17H21_iters59_remesh1_lambda1_0_searchk20_searchrad1_0_noise0_1_ntriangles264_duration4_5_ictm.stl
MSE : 467.37753175573977  iters: 59   remesh frequency: 1  lambda: 1.0   search k: 20   noise : 0.1  # triangles: 264 duration: 4.5
Best SPR mesh...
C:\Users\zrc4\Downloads\test_ictm_vs_psr\twj_h500_r50_smoothing50_20220601_17H21_searchk20_depth2_scale1_1_samplespernode13_0_pointweight3_0_iters7_noise0_1_duration0_1_spr.stl
MSE : 103876.88318395478  search k: 20   depth: 2   scale: 1.1   samples per node: 13.0   pointweight: 3.0   iters: 7   noise : 0.1  duration: 0.1
Data munged to vertices, faces
initializing halfedges ...
vertices.shape = (134, 3), faces.shape = (264, 3)
iterating edges
done initializing halfedges
lw update
Disposing of HistLimitsEditor
Creating layers GUI
hl update
hl update
height:  349
NB best size: wx.Size(819, 644)


ERROR:traitsui.wx.toolkit:Wx control <traitsui.wx.helper.TraitsUIPanel object at 0x0000022302601D38> not destroyed cleanly
Traceback (most recent call last):
  File "C:\Users\zrc4\Miniconda3\envs\pyme\lib\site-packages\traitsui\wx\toolkit.py", line 436, in _destroy_control
    control.Destroy()
  File "C:\Users\zrc4\Miniconda3\envs\pyme\lib\site-packages\traitsui\wx\helper.py", line 326, in Destroy
    self.Unbind(wx.EVT_CHILD_FOCUS)
  File "C:\Users\zrc4\Miniconda3\envs\pyme\lib\site-packages\wx\core.py", line 1416, in _EvtHandler_Unbind
    return event.Unbind(self, id, id2, handler)
  File "C:\Users\zrc4\Miniconda3\envs\pyme\lib\site-packages\wx\core.py", line 1485, in Unbind
    success += int(target.Disconnect(id1, id2, et, handler))
RuntimeError: wrapped C/C++ object of type TraitsUIPanel has been deleted
DEBUG:PYME.LMVis.shader_programs.ShaderProgramFactory:New shader program created: <class 'PYME.LMVis.shader_programs.GouraudShaderProgram.GouraudShaderProgram'>


Data munged to vertices, faces
initializing halfedges ...
vertices.shape = (43, 3), faces.shape = (61, 3)
iterating edges
done initializing halfedges
lw update
Disposing of HistLimitsEditor
Disposing of HistLimitsEditor
Creating layers GUI
hl update
hl update
hl update


ERROR:traitsui.wx.toolkit:Wx control <traitsui.wx.helper.TraitsUIPanel object at 0x000002230277E558> not destroyed cleanly
Traceback (most recent call last):
  File "C:\Users\zrc4\Miniconda3\envs\pyme\lib\site-packages\traitsui\wx\toolkit.py", line 436, in _destroy_control
    control.Destroy()
  File "C:\Users\zrc4\Miniconda3\envs\pyme\lib\site-packages\traitsui\wx\helper.py", line 326, in Destroy
    self.Unbind(wx.EVT_CHILD_FOCUS)
  File "C:\Users\zrc4\Miniconda3\envs\pyme\lib\site-packages\wx\core.py", line 1416, in _EvtHandler_Unbind
    return event.Unbind(self, id, id2, handler)
  File "C:\Users\zrc4\Miniconda3\envs\pyme\lib\site-packages\wx\core.py", line 1485, in Unbind
    success += int(target.Disconnect(id1, id2, et, handler))
RuntimeError: wrapped C/C++ object of type TraitsUIPanel has been deleted
ERROR:traitsui.wx.toolkit:Wx control <traitsui.wx.helper.TraitsUIPanel object at 0x0000022303AC2D38> not destroyed cleanly
Traceback (most recent call last):
  File "C:\Users\zr

height:  374
NB best size: wx.Size(836, 374)


...

ERROR:traits:Exception occurred in traits notification handler for object: <PYME.LMVis.layers.pointcloud.PointCloudRenderLayer object at 0x0000022302251E28>, trait: visible, old value: False, new value: True
Traceback (most recent call last):
  File "C:\Users\zrc4\Miniconda3\envs\pyme\lib\site-packages\traits\trait_notifiers.py", line 524, in _dispatch_change_event
    self.dispatch(handler, *args)
  File "C:\Users\zrc4\Miniconda3\envs\pyme\lib\site-packages\traits\trait_notifiers.py", line 486, in dispatch
    handler(*args)
  File "c:\users\zrc4\code\python-microscopy\PYME\LMVis\layers\pointcloud.py", line 139, in <lambda>
    self.on_trait_change(lambda: self.on_update.send(self), 'visible')
  File "c:\users\zrc4\code\python-microscopy\PYME\contrib\dispatch\dispatcher.py", line 174, in send
    for receiver in self._live_receivers(sender)
  File "c:\users\zrc4\code\python-microscopy\PYME\contrib\dispatch\dispatcher.py", line 174, in <listcomp>
    for receiver in self._live_receiver

hl update


ERROR:traits:Exception occurred in traits notification handler for object: <PYME.LMVis.layers.mesh.TriangleRenderLayer object at 0x00000223027D3A08>, trait: visible, old value: False, new value: True
Traceback (most recent call last):
  File "C:\Users\zrc4\Miniconda3\envs\pyme\lib\site-packages\traits\trait_notifiers.py", line 524, in _dispatch_change_event
    self.dispatch(handler, *args)
  File "C:\Users\zrc4\Miniconda3\envs\pyme\lib\site-packages\traits\trait_notifiers.py", line 486, in dispatch
    handler(*args)
  File "c:\users\zrc4\code\python-microscopy\PYME\ui\layerFoldPanel.py", line 118, in _refresh
    self.Refresh()
RuntimeError: wrapped C/C++ object of type LayerCaptionBar has been deleted


Data munged to vertices, faces
initializing halfedges ...
vertices.shape = (6986, 3), faces.shape = (13968, 3)
iterating edges
done initializing halfedges
lw update
Disposing of HistLimitsEditor
Disposing of HistLimitsEditor
Disposing of HistLimitsEditor
Creating layers GUI
hl update
hl update
hl update
hl update
height:  399
NB best size: wx.Size(836, 692)


ERROR:traitsui.wx.toolkit:Wx control <traitsui.wx.helper.TraitsUIPanel object at 0x00000223040D5318> not destroyed cleanly
Traceback (most recent call last):
  File "C:\Users\zrc4\Miniconda3\envs\pyme\lib\site-packages\traitsui\wx\toolkit.py", line 436, in _destroy_control
    control.Destroy()
  File "C:\Users\zrc4\Miniconda3\envs\pyme\lib\site-packages\traitsui\wx\helper.py", line 326, in Destroy
    self.Unbind(wx.EVT_CHILD_FOCUS)
  File "C:\Users\zrc4\Miniconda3\envs\pyme\lib\site-packages\wx\core.py", line 1416, in _EvtHandler_Unbind
    return event.Unbind(self, id, id2, handler)
  File "C:\Users\zrc4\Miniconda3\envs\pyme\lib\site-packages\wx\core.py", line 1485, in Unbind
    success += int(target.Disconnect(id1, id2, et, handler))
RuntimeError: wrapped C/C++ object of type TraitsUIPanel has been deleted
ERROR:traitsui.wx.toolkit:Wx control <traitsui.wx.helper.TraitsUIPanel object at 0x000002230543C0D8> not destroyed cleanly
Traceback (most recent call last):
  File "C:\Users\zr

In [ ]:
import matplotlib.pyplot as plt

def make_scatterplots(x, y, label_x = None, label_y = None, title=None):
    assert(len(x) == len(y))
    rows, cols = (len(x)+1)//2, 2
    fig, axs = plt.subplots(rows, cols)
    for i in range(len(x)):
        axs[i//2][i%2].scatter(x[i], y[i], s=0.1)
        if label_x:
            axs[i//2][i%2].set_xlabel(label_x[i])
        if label_y:
            axs[i//2][i%2].set_ylabel(label_y[i])
    if title:
        fig.suptitle(title, fontsize=16)
        
ictm_x = [ictm_its, ictm_remesh, ictm_lamb, ictm_searchk]
ictm_y = [ictm_mses]*len(ictm_x)
ictm_label_x = ['iterations', 'remesh frequency', 'lambda', 'search k']
ictm_label_y = ['MSE']*len(ictm_x)
make_scatterplots(ictm_x, ictm_y, ictm_label_x, ictm_label_y, 'ICTM')
        
spr_x = [spr_searchk, spr_depth, spr_scale, spr_spn, spr_pointweight, spr_its]
spr_y = [spr_mses]*len(spr_x)
spr_label_x = ['search k', 'depth', 'scale', 'samples per node', 'point weight', 'iterations']
spr_label_y = ['MSE']*len(spr_x)
make_scatterplots(spr_x, spr_y, spr_label_x, spr_label_y, 'SPR')

In [ ]:
import itertools
def make_comparative_plot(x, y, list_fixed=[], vals_fixed=None):
    """
    Compare x and y holding values in list_fixed fixed. vals_fixed can
    be used to make a subrange of list_fixed.
    """
    assert(len(x) == len(y))
    for it in list_fixed:
        assert(len(x) == len(it))
    unique_fixed = [np.unique(it) for it in list_fixed]
    if vals_fixed:
        vals_fixed = [it if len(jt) == 0 else jt for it, jt in zip(unique_fixed,vals_fixed)]
        unique_fixed = [list(set(it).intersection(jt)) for it, jt in zip(unique_fixed,vals_fixed)]
    opts = itertools.product(*unique_fixed)
    for opt in opts:
        idxs = np.ones(len(x),dtype=bool)
        i = 0
        for val in opt:
            idxs &= (list_fixed[i] == val)
            i += 1
        xi = x[idxs]
        srt = np.argsort(xi)
        plt.plot(xi[srt], y[idxs][srt])

plt.figure()
make_comparative_plot(ictm_its, np.array(ictm_mses), [ictm_lamb, ictm_remesh, ictm_searchk], [[], [50], [10]])
plt.legend([f'lambda={lam}' for lam in np.unique(ictm_lamb)])
plt.xlabel('# iterations')
plt.ylabel('MSE')